In [ ]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, OpenAIServerModel
import os
model = OpenAIServerModel(
    model_id="gpt-4o-mini",
    api_key=os.environ["OPENAI_API_KEY"],
)
agent = CodeAgent(tools=[DuckDuckGoSearchTool()], model=model)
agent.run("Who is Brandon Eychaner?")

In [1]:
"""
    CDG Client - An example client for the Congress.gov API.

    @copyright: 2022, Library of Congress
    @license: CC0 1.0
"""
from urllib.parse import urljoin
import requests
import logging 

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)


API_VERSION = "v3"
ROOT_URL = "https://api.congress.gov/"
RESPONSE_FORMAT = "json"


class _MethodWrapper:
    """ Wrap request method to facilitate queries.  Supports requests signature. """

    def __init__(self, parent, http_method):
        self._parent = parent
        self._method = getattr(parent._session, http_method)

    def __call__(self, endpoint, *args, **kwargs):  # full signature passed here
        response = self._method(
            urljoin(self._parent.base_url, endpoint), *args, **kwargs
        )
        if response.headers.get("content-type", "").startswith("application/json"):
            return response.json()
        else:
            return response.content

class CDGClient:
    """ A sample client to interface with Congress.gov. """

    def __init__(
        self,
        api_key,
        api_version=API_VERSION,
        response_format=RESPONSE_FORMAT,
        raise_on_error=True,
        added_headers=None
    ):
        self.base_url = urljoin(ROOT_URL, api_version) + "/"
        self._session = requests.Session()

        # do not use url parameters, even if offered, use headers
        self._session.params = {"format": response_format}
        self._session.headers.update({"x-api-key": api_key})
        if added_headers:
            self._session.headers.update(added_headers)

        if raise_on_error:
            self._session.hooks = {
                "response": lambda r, *args, **kwargs: r.raise_for_status()
            }

    def __getattr__(self, method_name):
        """Find the session method dynamically and cache for later."""
        method = _MethodWrapper(self, method_name)
        self.__dict__[method_name] = method
        return method

In [ ]:
from datetime import datetime
from enum import StrEnum
from pydantic import BaseModel, HttpUrl, Field, field_validator
from typing import List, Annotated, Optional

class Format(BaseModel):
    type: str
    url: HttpUrl
    full_text: str = "" # This needs to be retrieved from the url

class BehalfType(StrEnum):
    #This can be "Submitted on behalf of" the sponsor and/or "Proposed on behalf of" the sponsor. Assign value based on first word of input
    SUBMITTED = "Submitted"
    PROPOSED = "Proposed"

    def __init__(self, value):
        self._value_ = value
        self.type_url = value.split()[0]

class AmendmentType(StrEnum):
    SAMDT = "SAMDT" # Senate Amendment
    HAMDT = "HAMDT" # House Amendment
    SUAMDT = "SUAMDT" # Senate Unnumbered Amendment

    def __init__(self, value):
        self._value_ = value
        self.type_url = value.lower()

class TextVersion(BaseModel):
    date: datetime
    formats: List[Format]
    type: str

class PolicyArea(BaseModel):
    name: str
    update_date: Annotated[datetime, Field(alias='updateDate')] = None

class LegislativeSubject(BaseModel):
    name: str
    update_date: Annotated[datetime, Field(alias='updateDate')] = None

class Chamber(StrEnum):
    HOUSE = "House"
    SENATE = "Senate"

class LawType(StrEnum):
    PUBLIC = "Public Law"
    PRIVATE = "Private Law"

    def __init__(self, value):
        self._value_ = value
        if value == "Public Law":
            self.type_url = "pub"
        elif value == "Private Law":
            self.type_url = "priv"

class LatestAction(BaseModel):
    action_date: Annotated[datetime, Field(alias="actionDate")] = None
    text: str

class Note(BaseModel):
    texts: Annotated[List[str], Field(alias='text')]
    text: str = ""
    def __init__(self, text: str):
        self.text = ["\n".join(x["text"]) for x in text]

class Summary(BaseModel):
    action_date: Annotated[datetime, Field(alias='actionDate')]
    action_desc: Annotated[str, Field(alias='actionDesc')]
    text: str
    updateDate: Annotated[datetime, Field(alias='updateDate')]
    version_code: Annotated[str, Field(alias='versionCode')]

class CommitteeMetadata(BaseModel):
    name: str
    system_code: Annotated[str, Field(alias='systemCode')]
    url: HttpUrl
    
class SourceSystem(BaseModel):
    name: str
    code: int = -1

class ActionSourceSystem(StrEnum):
    SENATE = "0"
    LIBRARY_OF_CONGRESS = "9"
    HOUSE1 = "1"
    HOUSE2 = "2"

class Activity(BaseModel):
    date: datetime
    name: str

class IdentifyingEntity(StrEnum):
    HOUSE = "House"
    SENATE = "Senate"
    CRS = "CRS" # Congressional Research Service

class RelationshipDetail(BaseModel):
    identified_by: Annotated[IdentifyingEntity, Field(alias='identifiedBy')] = None
    type: str

class CountUrl(BaseModel):
    count: int
    url: HttpUrl

class Title(BaseModel):
    title: str
    title_type: Annotated[str, Field(alias='titleType')]
    title_type_code: Annotated[int, Field(alias='titleTypeCode')]
    update_date: Annotated[datetime, Field(alias='updateDate')]
    bill_text_version_code: Annotated[str, Field(alias='billTextVersionCode')] = ""
    bill_text_version_name: Annotated[str, Field(alias='billTextVersionName')] = ""

class ChamberCode(StrEnum):
    house = "H"
    senate = "S"

class BillType(StrEnum):
    def __init__(self, value):
        self._value_ = value
        self.type_url = value.lower()
    
    HR = "HR" # Bill introduced in House
    S = "S" # Bill introduced in Senate
    HJRES = "HJRES" # Joint resolution introduced in House
    SJRES = "SJRES" # Joint resolution introduced in Senate
    HCONRES = "HCONRES" # Concurrent resolution introduced in House
    SCONRES = "SCONRES" # Concurrent resolution introduced in Senate
    HRES = "HRES" # Simple resolution introduced in House
    SRES = "SRES" # Simple resolution introduced in Senate

class Member(BaseModel):
    bioguide_id: Annotated[str, Field(alias='bioguideId')]
    firstName: str
    first_name: Annotated[str, Field(alias='firstName')]
    full_name: Annotated[str, Field(alias='fullName')]
    last_name: Annotated[str, Field(alias='lastName')]
    party: str
    state: str
    url: HttpUrl
    middle_name: Annotated[str, Field(alias='middleName')] = ""
    district: Optional[int] = None
    is_original_cosponsor: Annotated[bool, Field(alias='isOriginalCosponsor')] = False
    is_by_request: Annotated[str, Field(alias='isByRequest')] = ""

class Sponsor(Member):
    sponsorship_date: Annotated[datetime, Field(alias='sponsorshipDate')]
    is_original_cosponsor: Annotated[bool, Field(alias='isOriginalCosponsor')] = False
    sponsorship_withrawn_date: Annotated[datetime, Field(alias='sponsorshipWithdrawnDate')] = None


class LawMetadata(BaseModel):
    number: str
    law_type: Annotated[LawType, Field(alias="type")]

    @field_validator('law_type', mode='before')
    def convert_law_type(cls, value):
        if value == "Public Law":
            return LawType.PUBLIC
        elif value == "Private Law":
            return LawType.PRIVATE
        raise ValueError("Invalid law type")

class Law(BaseModel):
    congress: int
    latest_action: Annotated[LatestAction, Field(alias="latestAction")]
    laws: List[LawMetadata]
    number: str
    origin_chamber: Annotated[Chamber, Field(alias="originChamber")]
    origin_chamber_code: Annotated[ChamberCode, Field(alias="originChamberCode")]
    title: str
    bill_type: Annotated[BillType, Field(alias="type")]
    update_date: Annotated[datetime, Field(alias="updateDate")]
    update_date_including_text: Annotated[datetime, Field(alias="updateDateIncludingText")]
    url: HttpUrl

class Committee(BaseModel):
    activities: List[Activity]
    chamber: str
    name: str
    system_code: Annotated[str, Field(alias='systemCode')]
    type: str
    url: HttpUrl

class RecordedVote(BaseModel):
    roll_number: Annotated[int, Field(alias='rollNumber')]
    url: HttpUrl
    chamber: Annotated[Chamber, Field(alias='chamber')]
    congress: int
    date: datetime
    session_number: Annotated[int, Field(alias='sessionNumber')]

class Action(BaseModel):
    action_date: Annotated[datetime, Field(alias='actionDate')]
    committees: Optional[List[CommitteeMetadata]] = []
    source_system: Annotated[SourceSystem, Field(alias='sourceSystem')] = None
    text: str
    type: str
    action_code: Annotated[str, Field(alias='actionCode')] = ""
    action_time: Annotated[datetime, Field(alias='actionTime')] = None
    recorded_votes: Annotated[List[RecordedVote], Field(alias='recordedVotes')] = []

class AmendmentMetadata(BaseModel):
    congress: int
    latest_action: Annotated[LatestAction, Field(alias='latestAction')] = None
    number: str
    purpose: str = ""
    type: str
    update_date: Annotated[datetime, Field(alias='updateDate')]
    url: HttpUrl

class Treaty(BaseModel):
    congress: int
    treaty_number: Annotated[str, Field(alias='treatyNumber')]
    url: HttpUrl

class Amendment(BaseModel):
    congress: int
    description: str
    purpose: str = ""
    latest_action: Annotated[LatestAction, Field(alias='latestAction')] = None
    number: str
    type: str
    update_date: Annotated[datetime, Field(alias='updateDate')]
    url: HttpUrl
    sponsors: List[Member] = []
    cosponsors: List[Member] = [] # Note that this needs to be populated in a separate step
    on_behalf_of_sponsor: Annotated[Member, Field(alias='onBehalfOfSponsor')] = None
    behalf_type: Annotated[BehalfType, Field(alias='behalfType')] = None
    proposed_date: Annotated[datetime, Field(alias='proposedDate')] = None
    submitted_date: Annotated[datetime, Field(alias='submittedDate')] = None
    chamber: Annotated[Chamber, Field(alias='chamber')] = None
    amended_treaty: Annotated[Treaty, Field(alias='amendedTreaty')] = None
    actions: List[Action] = [] # This needs to be populated in a separate step

class BillMetadata(BaseModel):
    congress: int
    latest_action: Annotated[LatestAction, Field(alias="latestAction")]
    number: int
    relationship_details: Annotated[List[RelationshipDetail], Field(alias="relationshipDetails")] = []
    title: str
    type: str
    url: HttpUrl

class Subjects(BaseModel):
    legislative_subjects: Annotated[List[LegislativeSubject], Field(alias='legislativeSubjects')] = []
    policy_area: Annotated[PolicyArea, Field(alias='policyArea')]

class Bill(BaseModel):
    congress: int
    constitutional_authority_statement_text: Annotated[str, Field(alias="constitutionalAuthorityStatement")] = ""
    introduced_date: Annotated[datetime, Field(alias="introducedDate")] = None
    latest_action: Annotated[LatestAction, Field(alias="latestAction")]
    laws: Annotated[List[LawMetadata], Field(alias="laws")] = []
    number: str
    origin_chamber: Annotated[Chamber, Field(alias="originChamber")]
    origin_chamber_code: Annotated[ChamberCode, Field(alias="originChamberCode")]
    policy_area: Annotated[PolicyArea, Field(alias="policyArea")] = None
    sponsors: List[Member] = []
    title: str
    type: str
    update_date: Annotated[datetime, Field(alias="updateDate")]
    update_date_including_text: Annotated[datetime, Field(alias="updateDateIncludingText")]
    notes: Annotated[Note, Field(alias="notes")] = None


In [112]:
client = CDGClient(api_key=os.environ["CONGRESS_API_KEY"], response_format=RESPONSE_FORMAT)

In [127]:
from urllib.parse import urljoin, urlparse, parse_qs
import requests
import logging
import os
import time
from tqdm import tqdm

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

API_VERSION = "v3"
ROOT_URL = "https://api.congress.gov/"
RESPONSE_FORMAT = "json"
RESULT_LIMIT = 250
RATE_LIMIT_CONSTANT = 5000 / 60 / 60  # 5000 requests per hour

class _MethodWrapper:
    """ Wrap request method to facilitate queries.  Supports requests signature. """

    def __init__(self, parent, http_method):
        self._parent = parent
        self._method = getattr(parent._session, http_method)

    def __call__(self, endpoint, *args, **kwargs):  # full signature passed here
        response = self._method(
            urljoin(self._parent.base_url, endpoint), *args, **kwargs
        )
        if response.headers.get("content-type", "").startswith("application/json"):
            return response.json()
        else:
            return response.content

class CDGClient:
    """ A sample client to interface with Congress.gov. """

    def __init__(
        self,
        api_key,
        api_version=API_VERSION,
        response_format=RESPONSE_FORMAT,
        raise_on_error=True,
        added_headers=None
    ):
        self.base_url = urljoin(ROOT_URL, api_version) + "/"
        self._session = requests.Session()

        # do not use url parameters, even if offered, use headers
        self._session.params = {"format": response_format}
        self._session.headers.update({"x-api-key": api_key})
        if added_headers:
            self._session.headers.update(added_headers)

        if raise_on_error:
            self._session.hooks = {
                "response": lambda r, *args, **kwargs: r.raise_for_status()
            }

    def __getattr__(self, method_name):
        """Find the session method dynamically and cache for later."""
        method = _MethodWrapper(self, method_name)
        self.__dict__[method_name] = method
        return method

    def get_bills_metadata(self, from_date: str, to_date: str, offset: int = 0):
        params = {
            "limit": RESULT_LIMIT,
            "fromDateTime": from_date,
            "toDateTime": to_date
        }
        
        if offset > 0:
            params["offset"] = offset

        response = self.get("bill", params=params)
        bills = response.get("bills", [])
        if "next" in response.get("pagination", {}):
            offset = self.extract_offset(response["pagination"]["next"])
            return (bills, response, offset, response["pagination"]["count"])
        return (bills, response, -1, 0)
    
    def get_bill_details(self, bill_metadata: BillMetadata) -> Bill:
        """
        Retrieve additional data for a bill.

        Args:
            client (CDGClient): The client object.
            bill_data (BillMetadata): The bill metadata.

        Returns:
            Bill: The bill object.
        """
        bill_data = {}
        # Currently available endpoints for additional data on bills
        additional_bill_data = {
            'actions': 'actions',
            'amendments': 'amendments',
            'committees': 'committees',
            'cosponsors': 'cosponsors',
            'relatedBills': 'relatedbills',
            'subjects': 'subjects',
            'summaries': 'summaries',
            'textVersions': 'text',
            'titles': 'titles'
        }
        congress = bill_metadata.congress
        bill_type = bill_metadata.type.lower()
        bill_number = bill_metadata.number
        for key, endpoint in additional_bill_data.items():
            data = self.get(f"bill/{congress}/{bill_type}/{bill_number}/{endpoint}")
            bill_data[key] = data[key]
        bill_data.update(bill_metadata.model_dump())
        return bill_data
    
    def get_laws_metadata(self, start_date: str, end_date: str, congress: int, offset: int = 0):
        params = {
            "fromDateTime": start_date,
            "toDateTime": end_date
        }
        if offset > 0:
            params["offset"] = offset
        response = self.get(f"law/{congress}", params=params)
        laws = response.get("laws", [])
        if "next" in response.get("pagination", {}):
            offset = self.extract_offset(response["pagination"]["next"])
            return (laws, offset, response["pagination"]["count"])
        return (laws, -1, 0)
    
    def get_law_details(self, law_metadata: LawMetadata) -> Law:
        """
        Retrieve additional data for a law.

        Args:
            client (CDGClient): The client object.
            law_metadata (LawMetadata): The law metadata.

        Returns:
            Law: The law object.
        """
        law_data = {}
        congress = law_metadata.congress
        law_type = law_metadata.type.lower()
        law_number = law_metadata.number
        additional_law_data = {
            'actions': 'actions',
            'amendments': 'amendments',
            'cosponsors': 'cosponsors',
            'textVersions': 'text'
        }
        for key, endpoint in additional_law_data.items():
            data = self.get(f"law/{congress}/{law_type}/{law_number}/{endpoint}")
            law_data[key] = data[key]
        law_data.update(law_metadata.model_dump())
        law = Law(**law_data)
        return law
    
    def get_amendments_metadata(self, bill_number: int, congress: int, amendment_type: AmendmentType):
        response = self.get(f"amendment/{congress}/{amendment_type.type_url}/{bill_number}")

        return response
    
    def get_amendment_details(self, amendment_metadata: AmendmentMetadata) -> Amendment:
        """
        Retrieve additional data for an amendment.

        Args:
            client (CDGClient): The client object.
            amendment_metadata (AmendmentMetadata): The amendment metadata.

        Returns:
            Amendment: The amendment object.
        """
        amendment_data = {}
        congress = amendment_metadata.congress
        amendment_type = amendment_metadata.type
        amendment_number = amendment_metadata.number
        additional_amendment_data = {
            'actions': 'actions',
            'amendedBill': 'bill',
            'cosponsors': 'cosponsors', 
            'text': 'textVersions'
        }

        for key, endpoint in additional_amendment_data.items():
            data = self.get(f"amendment/{congress}/{amendment_type.type_url}/{amendment_number}/{endpoint}")
            amendment_data[key] = data[key]
        amendment_data.update(amendment_metadata.model_dump())
        amendment = Amendment(**amendment_data)
        return amendment
        
    def extract_offset(self, url: str) -> int:
        parsed_url = urlparse(url)
        offset = parse_qs(parsed_url.query).get('offset', [0])[0]
        return int(offset)

def determine_pagination_wait(start_time: float, offset: int):
    """
    Determine the wait time based on the rate limit constant.
    
    Args:
        start_time (float): The start time of the request.
        offset (int): The offset for the request.
        
    Returns:
        None
    """
    current_time = time.time()
    elapsed_time = current_time - start_time
    print(f"Elapsed time: {elapsed_time}")
    requests = max(RESULT_LIMIT, offset) / RESULT_LIMIT
    rate = elapsed_time / requests
    if rate < RATE_LIMIT_CONSTANT:
        wait_time = RATE_LIMIT_CONSTANT - rate
        print(f"Sleeping for {wait_time} seconds.")
        time.sleep(wait_time)


def determine_simple_wait(start_time: float, api_call_count: int):
    """
    Determine the wait time based on the rate limit constant.

    Args:
        start_time (float): The start time of the request.
        api_call_count (int): The number of API calls made.
    """
    current_time = time.time()
    elapsed_time = current_time - start_time
    rate = elapsed_time / api_call_count
    if rate < RATE_LIMIT_CONSTANT:
        wait_time = RATE_LIMIT_CONSTANT - rate
        print(f"Sleeping for {wait_time} seconds.")
        time.sleep(wait_time)

def gather_congress_bills(client: CDGClient, from_date: str, to_date: str) -> list:
    """
    Gather all bills for a given date range.

    Args:
        client (CDGClient): The client object.
        from_date (str): The start date for the search in the format "YYYY-MM-DDTHH:MM:SSZ".
        to_date (str): The end date for the search in the format "YYYY-MM-DDTHH:MM:SSZ".

    Returns:
        list: A list of bill metadata
    """
    start = time.time()
    bills = []
    offset = 0
    total_count = None
    pbar = None

    while offset != -1:
        result, offset, count = client.get_bills_metadata(from_date, to_date, offset)
        bills.extend(result)
        if total_count is None:
            total_count = count
            pbar = tqdm(total=total_count, desc="Retrieving bills")
        pbar.update(len(result))
        determine_pagination_wait(start, offset)  # Prevent rate limiting
    if pbar:
        pbar.close()
    return bills


In [ ]:
# Usage
client = CDGClient(api_key=os.environ["CONGRESS_API_KEY"], response_format=RESPONSE_FORMAT)
bills = gather_congress_bills(client, "2024-02-15T00:00:00Z", "2024-03-30T00:00:00Z")

In [ ]:
client = CDGClient(api_key=os.environ["CONGRESS_API_KEY"], response_format=RESPONSE_FORMAT)
bill_data = []
api_hit_count = 0
start = time.time()
for i, bill in enumerate(bills[:1000]):
    # Progress
    if i % 10 == 0:
        print(f"{i}/1000")
    bill_data.append(client.get_bill_details(bill))
    api_hit_count += 10
    determine_simple_wait(start, api_hit_count)  # Prevent rate limiting

In [73]:
# Save bills to file 
import json
from datetime import datetime

def save_bills(bills: list[dict], filename: str):
    with open(filename, "w") as f:
        json.dump(bills, f)

def load_bills(filename: str) -> list[dict]:
    with open(filename, "r") as f:
        return json.load(f)
    
save_bills(bills, f"bills_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.json")


In [121]:
load_bills("bills_2025-02-28_14-27-13.json")[0].keys()

dict_keys(['congress', 'latestAction', 'number', 'originChamber', 'originChamberCode', 'title', 'type', 'updateDate', 'updateDateIncludingText', 'url', 'actions', 'amendments', 'committees', 'cosponsors', 'relatedBills', 'subjects', 'summaries', 'textVersions', 'titles'])

In [90]:
def save_additional_data(data: list, filename: str):
    with open(filename, "w") as f:
        json.dump(data, f)

def load_additional_data(filename: str) -> list:
    with open(filename) as f:
        data = json.load(f)
    return data

save_additional_data(bill_data, "additional_data.json")

In [114]:
# Try to load a bill, and fill the metadata
bill_metadata = BillMetadata(**bills[0])

In [132]:
client = CDGClient(api_key=os.environ["CONGRESS_API_KEY"], response_format=RESPONSE_FORMAT)
bill_data = client.get_bill_details(bill_metadata)

In [133]:
bill_data.keys()

dict_keys(['actions', 'amendments', 'committees', 'cosponsors', 'relatedBills', 'subjects', 'summaries', 'textVersions', 'titles', 'congress', 'latest_action', 'number', 'relationship_details', 'title', 'type', 'url'])

In [ ]:
# Find bill later with non-empty ["amendements", "summaries", "relationshipDetails"]

In [179]:
bill_data["congress"]

118

In [ ]:
actions = [Action(**x) for x in bill_data["actions"]]
amendments = [AmendmentMetadata(**x) for x in bill_data["amendments"]]
cosponsors = [Sponsor(**x) for x in bill_data["cosponsors"]]
related_bills = [BillMetadata(**x) for x in bill_data["relatedBills"]]
subjects = Subjects(**bill_data["subjects"])
summaries = [Summary(**x) for x in bill_data["summaries"]]
text_versions = [TextVersion(**x) for x in bill_data["textVersions"]] # TODO: Write function to ill in full_text field
titles = [Title(**x) for x in bill_data["titles"]]
latest_action = LatestAction(**bill_data["latest_action"]) # Fix inheritance. LatestAction can't take updated key from BillMetadata ("latest_action")
relationship_details = [RelationshipDetail(**x) for x in bill_data["relationship_details"]]


In [191]:
relationship_details

[]

In [195]:
bill_data["type"]

'SJRES'

In [ ]:
extra_data = []

def calculate_simple_wait(start_time: float, api_hit_count: int) -> float:
    current_time = time.time()
    elapsed_time = current_time - start_time
    rate = elapsed_time / api_hit_count
    if rate < RATE_LIMIT_CONSTANT:
        wait_time = RATE_LIMIT_CONSTANT - rate
        print(f"Sleeping for {wait_time} seconds.")
        time.sleep(wait_time)

api_hit_count = 0
for bill in bills:
    start = time.time()
    api_hit_count += 1
    bill_data = get_bill_data(client, bill)
    extra_data.append(bill_data)

{'congress': 118,
 'latestAction': {'actionDate': '2023-10-18',
  'text': 'Referred to the Committee on the Judiciary. (text: CR S5090-5091)'},
 'number': '413',
 'originChamber': 'Senate',
 'originChamberCode': 'S',
 'title': 'A resolution condemning foreign nationals in the United States who have endorsed and espoused the actions of foreign terrorist organizations (FTO) in Gaza who, on October 7, 2023, launched attacks against the State of Israel, and killed innocent Israeli and United States citizens.',
 'type': 'SRES',
 'updateDate': '2024-03-14',
 'updateDateIncludingText': '2024-03-14',
 'url': 'https://api.congress.gov/v3/bill/118/sres/413?format=json',
 'actions': [{'actionDate': '2023-10-18',
   'committees': [{'name': 'Judiciary Committee',
     'systemCode': 'ssju00',
     'url': 'https://api.congress.gov/v3/committee/senate/ssju00?format=json'}],
   'sourceSystem': {'name': 'Senate'},
   'text': 'Referred to the Committee on the Judiciary. (text: CR S5090-5091)',
   'type':